# Model data

*If a local variable 't' has not been defined, this runs the notebook locally; otherwise, it is assumed to be run from ModelData.ipynb.*

In [1]:
try:
    t = t
except NameError:
    clean_up = True
    %run StdPackages.ipynb
    t = 2018
    name = 'm'
error = 1e-7

The file grids.gdx is still active and was not deleted.
The file rname.gdx is still active and was not deleted.
The file _gams_py_gdb0.gdx is still active and was not deleted.
The file _gams_py_gdb1.gdx is still active and was not deleted.
The file _gams_py_gdb13.gdx is still active and was not deleted.
The file _gams_py_gdb16.gdx is still active and was not deleted.
The file _gams_py_gdb17.gdx is still active and was not deleted.
The file _gams_py_gdb18.gdx is still active and was not deleted.
The file _gams_py_gdb3.gdx is still active and was not deleted.
The file _gams_py_gdb4.gdx is still active and was not deleted.


Load data:

In [2]:
db = GpyDB(pickle_path = os.path.join(d['processedData'], f'IO{t}'))
db.name = f'{name}_{t}'
with open(os.path.join(d['processedData'],f'glob_{t}'), "rb") as file:
    glob = pickle.load(file)

Add real interest rate (the time dimension is added later):

In [3]:
db['Rrate'] = glob.db['R_LR'].vals

Next, run through the various modules and create relevant symbols used throughout:

## 1. Production module

### 1.1. Nesting structure

Energy inputs:

In [4]:
E= '35000_'
E_F = E+'_F'

Specify a general mapping that we'll apply to (most) production sectors:

In [5]:
mFull = pd.MultiIndex.from_tuples([('KELBM', 'RxE'), ('KELBM', 'KELB'), ('KELB', 'iB'), ('KELB','KEL'), ('KEL','L'), ('KEL','KE'), ('KE','iM'), ('KE','E'), ('E', f'{E}_input'), ('E',f'{E_F}')], names = ['n','nn'])

If a sector does not demand either E or E_F, remove the KE nest:

In [6]:
s_E = adj.rc_pd(db.get('vD'), pd.Index([E], name = 'n')).droplevel('n').index.unique()
s_EF= adj.rc_pd(db.get('vD'), pd.Index([E_F], name = 'n')).droplevel('n').index.unique()
m_noE = pd.MultiIndex.from_tuples([('KELBM', 'RxE'), ('KELBM', 'KELB'), ('KELB', 'iB'), ('KELB','KEL'), ('KEL','L'), ('KEL','iM')], names = ['n','nn'])
s_noE = adj.rc_pd(db.get('s_p'), ('not', s_E.union(s_EF)))
m = pyDatabases.cartesianProductIndex([s_noE, m_noE])

If a sector only demands E, and not E_F replace the lower nest with:

In [7]:
m_noEF = pd.MultiIndex.from_tuples([('KELBM', 'RxE'), ('KELBM', 'KELB'), ('KELB', 'iB'), ('KELB','KEL'), ('KEL','L'), ('KEL','KE'), ('KE','iM'), ('KE',f'{E}_input')], names = ['n','nn'])
s_noEF = adj.rc_pd(db.get('s_p'), s_E.difference(s_EF))
m = m.union(pyDatabases.cartesianProductIndex([s_noEF, m_noEF]))

If a sector only demands EF and not E:

In [8]:
m_noEdom = pd.MultiIndex.from_tuples([('KELBM', 'RxE'), ('KELBM', 'KELB'), ('KELB', 'iB'), ('KELB','KEL'), ('KEL','L'), ('KEL','KE'), ('KE','iM'), ('KE',E_F)], names = ['n','nn'])
s_noEdom = adj.rc_pd(db.get('s_p'), s_EF.difference(s_E))
m = m.union(pyDatabases.cartesianProductIndex([s_noEF, m_noEF]))

All sectors use both ```iB``` and ```iM``` - so, we finally check for whether all sectors demand labor:

In [9]:
m_noL = pd.MultiIndex.from_tuples([('KELBM', 'RxE'), ('KELBM', 'KELB'), ('KELB', 'iB'), ('KELB','KE'), ('KE','iM'), ('KE','E'), ('E', f'{E}_input'), ('E',f'{E_F}')], names = ['n','nn'])
s_noL = adj.rc_pd(db.get('s_p'), ('not', db.get('vD').xs('L',level='n')))
m = m.union(pyDatabases.cartesianProductIndex([s_noL, m_noL]))

For all other sectors, use the full nesting structure:

In [10]:
sFull = adj.rc_pd(db.get('s_p'), ('not', ('or', [s_noE, s_noEF, s_noEdom, s_noL])))
m = m.union(pyDatabases.cartesianProductIndex([sFull, mFull]))

Replace the upper-most level (KELBM) with the name of the sector:

In [11]:
df = m.to_frame(index=False)
df.loc[df.n == 'KELBM','n'] = df.loc[df.n == 'KELBM', 's']
m = pd.MultiIndex.from_frame(df)

**Materials nest**

The materials nest is slightly different depending on whether or not a given sector demands both the domestic and foreign version of a material. We split into three categories:
* For [s,n] in ```dImport[s,n,nn]```, we define a mapping from ```RxE``` to an intermediate good ```RxEym_x``` that is again an aggregate of the domestic/foreign goods ```x,x_F```. 
* For [s,n] in ```dImport_dom[s,n]```, we define a mapping from ```RxE``` to each of the domestic goods in ```n```; these are the sectors that only demand the domestic type of the good.
* For [s,n] in ```dImport_for[s,n]```, we define a mapping from ```RxE``` to each of the foreign good types in ```n```; these are the sectors that only demand the foreign type of the good.

*Remove energy nest - this is included elsewhere in the nesting structure:*

In [12]:
dImport = adj.rc_pd(db.get('dImport'), ('not', pd.Index([E], name = 'n')))
dImport_dom = adj.rc_pd(db.get('dImport_dom'), ('not', pd.Index([E], name = 'n')))
dImport_for = adj.rc_pd(db.get('dImport_for'), ('not', pd.Index([E_F], name = 'n')))

*Add mappings:*

In [13]:
df = adj.rc_pd(dImport, db.get('s_p')).to_frame(index=False).assign(m= lambda x: 'RxEym_'+x['n'].astype(str), u = 'RxE', n = lambda x: x['n'].astype(str)+'_input')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','m']]).rename(['s','n','nn']))
m = m.union(pd.MultiIndex.from_frame(df[['s','m','n']]).rename(['s','n','nn']))
m = m.union(pd.MultiIndex.from_frame(df[['s','m','nn']]).rename(['s','n','nn']))

*Domestic only*

In [14]:
df = adj.rc_pd(dImport_dom, db.get('s_p')).to_frame(index=False).assign(u = 'RxE', n = lambda x: x['n'].astype(str)+'_input')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn']))

*Foreign goods only:*

In [15]:
df = adj.rc_pd(dImport_for, db.get('s_p')).to_frame(index=False).assign(u = 'RxE')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn']))

Map de stachio:

In [16]:
db['nestProduction'] = m

### 1.2. Elasticities

Elasticity of substitution between materials and ```qKELB```

In [17]:
sigmaOut = pd.Series(0.5, index = m[m.get_level_values('s')==m.get_level_values('n')].droplevel('nn').unique(), name = 'sigma')

Elasticity of substitution ```KELB``` nest:

In [18]:
sigmaKELB = pd.Series(0.5, index  = m[m.get_level_values('n')=='KELB'].droplevel('nn').unique(), name = 'sigma')

Elasticity of substitution ```KEL``` nest:

In [19]:
sigmaKEL = pd.Series(0.6, index  = m[m.get_level_values('n')=='KEL'].droplevel('nn').unique(), name = 'sigma')

Elasticity of substitution ```KE``` nest:

In [20]:
sigmaKE = pd.Series(0.5, index  = m[m.get_level_values('n')=='KE'].droplevel('nn').unique(), name = 'sigma')

Elasticity of substitution ```E``` nest (import/domestic):

In [21]:
sigmaE = pd.Series(2, index  = m[m.get_level_values('n')=='E'].droplevel('nn').unique(), name = 'sigma')

Elasticity of substitution between materials:

In [22]:
sigmaRxE = pd.Series(0.5, index = m[m.get_level_values('n')=='RxE'].droplevel('nn').unique(), name = 'sigma')

Elasticity of substitution import/domestic types:

In [23]:
sigmaRxEym = pd.Series(2, index = m[m.get_level_values('n').str.contains('RxEy')].droplevel('nn').unique(), name = 'sigma')

Add to database:

In [24]:
sigma = pd.concat([sigmaOut, sigmaKELB, sigmaKEL, sigmaKE, sigmaE, sigmaRxE, sigmaRxEym], axis = 0 )
gpyDB.add_or_merge_vals(db, sigma, name = 'sigma')

### 1.2. Regulation and emissions

Define relevant sector outputs and output shares (relevant if sectors have multiple outputs):

In [25]:
output = adj.rc_pd(db['qS'], db['s_p']) # output
inputs = adj.rc_pd(db['qD'], db['s_p']) # inputs
outShares = output/pyDatabases.pdSum(output, 'n') # output shares
db['qCO2'] = db.get('qCO2') * outShares # overwrite qCO2 to be split into potentially multiple outputs from each sector

Redefine emissions as a share of the aggregate 1990 level:

In [26]:
db['qCO2'] = db.get('qCO2') / glob.db['M1990'].vals

Define implicit tax rate. With normalized emissions to 1990 level, the tax rate now defines the cost of increasing emissions by 1 unit of 1990 emissions (that's a lot). As we do not have emission tax data for $t<1995$, we use the year $1995$ to do this:

In [27]:
if t>=1995:
    db['tauCO2'] = adjMultiIndex.applyMult((db.get('vTax').xs('Emissions',level='taxTypes') / db.get('qCO2')).fillna(0),
                                       output.index)
else:
    db1995 = GpyDB(pickle_path = os.path.join(d['processedData'], f'{name}_1995'))
    db['tauCO2'] = db1995['tauCO2']
    # add the CO2 tax to the variable vTax:
    os.chdir(d['py'])
    from loadIO import addLevelToMultiIndex
    os.chdir(d['curr'])
    gpyDB.add_or_merge_vals(db, 
                            addLevelToMultiIndex( pyDatabases.pdSum((db.get('qCO2') * db.get('tauCO2')).dropna(), 'n'), 
                                                 'taxTypes','Emissions').reorder_levels(['s','taxTypes'])
                            , name = 'vTax')

Create a few dummies:

In [28]:
db['dtauCO2'] = db.get('tauCO2').index
db['dqCO2'] = db.get('qCO2').index

Add the rest of the regulation:
* Zero lump-sum taxes, zero taxes on inputs.
* Environmental taxes as per IO emisssions data,
* non-environmental output taxes adjusted to target total taxes

In [29]:
gpyDB.add_or_merge_vals(db, adjMultiIndex.applyMult(db.get('TotalTax'), output.index) / output, name = 'tauS') # tauS
gpyDB.add_or_merge_vals(db, pd.Series(0, index = inputs.index), name = 'tauD') # tauD
gpyDB.add_or_merge_vals(db, pd.Series(0, index = db.get('s_p'), name = 'tauLump'), name = 'tauLump') # tauLump
gpyDB.add_or_merge_vals(db, adj.rc_pd(db.get('tauS'), output)-db.get('vTax').xs('Emissions',level='taxTypes') / output, name = 'tauNonEnv')
gpyDB.add_or_merge_vals(db, adj.rc_pd(db.get('tauS'), output)-db.get('vTax').xs('Emissions',level='taxTypes') / output, name = 'tauNonEnv0')

Define emission targets:

In [30]:
tE= glob.db['tE'].vals[0]
os_v1 = glob.db['qOS_V1'].vals/glob.db['M1990'].vals
os_v2 = glob.db['qOS_V2'].vals/glob.db['M1990'].vals
Mnorm = glob.db['MData'].vals / glob.db['M1990'].vals
if t<2005:
    target_FP = pd.Series(0.8, index = pd.Index(range(2005, tE+1), name = 't'))
    target_V1 = pd.Series(np.linspace(1, 0, 76), index = pd.Index(range(1990, 2066), name = 't'))
    target_V2 = target_V1
elif t<2011:
    target_FP = pd.Series(0.79, index = pd.Index(range(2011, tE+1), name = 't'))
    target_V1 = pd.concat([pd.Series(np.linspace(0.8, 0.79, 7), index = pd.Index(range(2005, 2012), name = 't')).iloc[0:-1],
                           pd.Series(np.linspace(0.79, 0.79-0.59 * 20/15, 60), index = pd.Index(range(2011, 2071), name = 't')),
                           pd.Series(0, index = pd.Index([2071], name = 't'))],
                          axis = 0)
    target_V2 = pd.concat([pd.Series(np.linspace(Mnorm.xs(2005), 0.79, 7), index = pd.Index(range(2005, 2012), name = 't')).iloc[0:-1],
                           pd.Series(np.linspace(0.79, 0.79-0.59 * 20/15, 60), index = pd.Index(range(2011, 2071), name = 't')),
                           pd.Series(0, index = pd.Index([2071], name = 't'))],
                          axis = 0)
elif t<2020:
    target_FP = pd.Series(0.6, index = pd.Index(range(2020, tE+1), name = 't'))
    target_V1 = pd.concat([pd.Series(np.linspace(0.79, 0.6, 10), index = pd.Index(range(2011,2021), name = 't')).iloc[0:-1], 
                           pd.Series(np.linspace(0.6, 0, 46), index = pd.Index(range(2020, 2066), name = 't'))],
                          axis = 0)
    target_V2 = pd.concat([pd.Series(np.linspace(Mnorm.xs(2011), 0.6, 10), index = pd.Index(range(2011,2021), name = 't')).iloc[0:-1], 
                           pd.Series(np.linspace(0.6, 0, 46), index = pd.Index(range(2020, 2066), name = 't'))],
                          axis = 0)
else:
    target_FP = pd.concat([pd.Series([0.3, 0], index = pd.Index([2030, 2045], name = 't')),
                           pd.Series(-.1, index = pd.Index(range(2050, tE+1), name = 't'))], 
                          axis = 0)
    target_V1 = pd.concat([pd.Series(np.linspace(0.6, 0.3, 11), index = pd.Index(range(2020, 2031), name = 't')).iloc[0:-1],
                           pd.Series(np.linspace(0.3, 0, 16), index = pd.Index(range(2030, 2046), name = 't')).iloc[0:-1],
                           pd.Series(np.linspace(0, -0.1, 6), index = pd.Index(range(2045, 2051), name = 't'))],
                          axis = 0)
    target_V2 = pd.concat([pd.Series(np.linspace(Mnorm.xs(2020), 0.3, 11), index = pd.Index(range(2020, 2031), name = 't')).iloc[0:-1],
                           pd.Series(np.linspace(0.3, 0, 16), index = pd.Index(range(2030, 2046), name = 't')).iloc[0:-1],
                           pd.Series(np.linspace(0, -0.1, 6), index = pd.Index(range(2045, 2051), name = 't'))],
                          axis = 0)
nTransitory = target_V2.index.max()-t-1
# targetOS_V1 = target_V1.loc[t+1:].add(-pd.Series(os_v1/22, index = pd.Index(range(t+1,t+23), name = 't')), fill_value=0) # linear elimination in 22 years
targetOS_V1 = auxiliary.eliminateOvershoot(target_V1.loc[t+23], t, 22, target_V1, os_v1).combine_first(target_V1.loc[t+1:])
targetOS_V2 = auxiliary.eliminateOvershoot(target_V2.iloc[-1], t, nTransitory, target_V2, os_v2)
db['MTarget_FP'] = target_FP
db['MTarget_NO_V1'] = auxiliary.extrapolateUpper(target_V1.loc[t:], tE)
db['MTarget_NO_V2'] = auxiliary.extrapolateUpper(target_V2.loc[t:], tE)
db['MTarget_V1'] = auxiliary.extrapolateUpper(targetOS_V1, tE)
db['MTarget_V2'] = auxiliary.extrapolateUpper(targetOS_V2, tE)

Define sector-specific prices given regulation:

In [31]:
gpyDB.add_or_merge_vals(db, adj.rc_pd(db.get('p'), inputs)+adj.rc_pd(db.get('tauD'), inputs), name = 'pD')

## 2. Investment module

### 2.1. Nesting structure

The investment module consists only of the materials nest (including energy). Thus, it proceeds along the same lines:

In [32]:
dImport = adj.rc_pd(db.get('dImport'), db.get('s_i'))
dImport_dom = adj.rc_pd(db.get('dImport_dom'), db.get('s_i'))
dImport_for = adj.rc_pd(db.get('dImport_for'), db.get('s_i'))

*Add mappings:*

In [33]:
df = dImport.to_frame(index=False).assign(m = lambda x: 'Materials_'+x['n'].astype(str), u = 'Materials')
m = pd.MultiIndex.from_frame(df[['s','u','m']]).rename(['s','n','nn']) # mapping from materials to materials_x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','n']]).rename(['s','n','nn'])) # mapping from materials_x to x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','nn']]).rename(['s','n','nn'])) # mapping from materials_x to x_F

*Domestic only*

In [34]:
df = dImport_dom.to_frame(index=False).assign(u = 'Materials')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Foreign only:

In [35]:
df = dImport_for.to_frame(index=False).assign(u = 'Materials')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Replace the upper-most level (KELBM) with the name of the sector:

In [36]:
df = m.to_frame(index=False)
df.loc[df.n == 'Materials','n'] = df.loc[df.n == 'Materials', 's']
m = pd.MultiIndex.from_frame(df)

*Add nesting structure:*

In [37]:
db['nestInvestment'] = m

### 2.2: Elasticities:

In [38]:
sigmaI = pd.Series(0, index = m.droplevel('nn').unique(), name = 'sigma')
gpyDB.add_or_merge_vals(db, sigmaI, name = 'sigma')

### 2.2. Regulation:

The investment sector does not itself emit emissions; however, investments are still tied to emissions through this sectors' reliance on inputs from other sectors (that are emission intensive). The emission regulation is filled in as a "production module" as above (but with zero emissions), in order to rely on a similar code.

In [39]:
output = adj.rc_pd(db['qS'], db['s_i']) # output
inputs = adj.rc_pd(db['qD'], db['s_i']) # inputs
outShares = output/pyDatabases.pdSum(output, 'n') # output shares
gpyDB.add_or_merge_vals(db, (adj.rc_pd(db.get('qCO2'), db['s_i']) * outShares).fillna(0), name = 'qCO2')
gpyDB.add_or_merge_vals(db, pd.Series(0, index = output.index, name = 'tauCO2'), name = 'tauCO2')

Rest of the regulation part:

In [40]:
gpyDB.add_or_merge_vals(db, adjMultiIndex.applyMult(db.get('TotalTax'), output.index) / output, name = 'tauS') # tauS
gpyDB.add_or_merge_vals(db, pd.Series(0, index = inputs.index), name = 'tauD') # tauD
gpyDB.add_or_merge_vals(db, pd.Series(0, index = db.get('s_p'), name = 'tauLump'), name = 'tauLump') # tauLump
gpyDB.add_or_merge_vals(db, adj.rc_pd(db.get('tauS'), output)-db.get('vTax').xs('Emissions',level='taxTypes') / output, name = 'tauNonEnv')
gpyDB.add_or_merge_vals(db, adj.rc_pd(db.get('tauS'), output)-db.get('vTax').xs('Emissions',level='taxTypes') / output, name = 'tauNonEnv0')

Define sector-specific prices given regulation:

In [41]:
gpyDB.add_or_merge_vals(db, adj.rc_pd(db.get('p'), inputs)+adj.rc_pd(db.get('tauD'), inputs), name = 'pD')

## 3. Households

### 3.1. Nesting structure

The household consumption nesting structure is somewhat similar to the investment sectors, with the exception that the top nest is a consumption aggregate that captures intertemporal preferences for consumption smoothing.

In [42]:
dImport = adj.rc_pd(db.get('dImport'), db.get('s_HH'))
dImport_dom = adj.rc_pd(db.get('dImport_dom'), db.get('s_HH'))
dImport_for = adj.rc_pd(db.get('dImport_for'), db.get('s_HH'))

*Add mappings:*

In [43]:
df = dImport.to_frame(index=False).assign(m = lambda x: 'C_'+x['n'].astype(str), u = 'C')
m = pd.MultiIndex.from_frame(df[['s','u','m']]).rename(['s','n','nn']) # mapping from materials to materials_x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','n']]).rename(['s','n','nn'])) # mapping from materials_x to x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','nn']]).rename(['s','n','nn'])) # mapping from materials_x to x_F

*Domestic only*

In [44]:
df = dImport_dom.to_frame(index=False).assign(u = 'C')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Foreign only:

In [45]:
df = dImport_for.to_frame(index=False).assign(u = 'C')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Replace the upper-most level with the name of the household consumption aggregate:

In [46]:
df = m.to_frame(index=False)
df.loc[df.n == 'C','n'] = 'C_'+df.loc[df.n == 'C', 's']
m = pd.MultiIndex.from_frame(df)

*Add nesting structure:*

In [47]:
db['nestHH'] = m

Add mapping from consumption aggregate to labor:

In [48]:
db['L2C'] = pd.MultiIndex.from_arrays([db.get('s_HH'), pd.Index(['L']*len(db.get('s_HH')), name = 'n'), ('C_'+db.get('s_HH')).rename('nn')])

### 3.2. Elasticities:

Upper level nest:

In [49]:
sigma_HH_upper = pd.Series(0.3, index = m[m.get_level_values('n') == 'C_HH'].droplevel('nn').unique(), name = 'sigma')

Lower-level (import/domestic competition):

In [50]:
sigma_HH_Import = pd.Series(2, index = m[m.get_level_values('n') != 'C_HH'].droplevel('nn').unique(), name = 'sigma')

Add to database:

In [51]:
sigma_HH = pd.concat([sigma_HH_upper, sigma_HH_Import], axis = 0) 
gpyDB.add_or_merge_vals(db, sigma_HH, name = 'sigma')

### 3.2. Regulation:

Households do not directly emit anything (it all happens in the production process, at the moment). Currently, we use a very simple type of regulation here:
* Households pay VAT (the size of which is included in the IO data); we impose this as a flat rate on all inputs.
* Set the labor income tax rate at 0.47 (roughly the average for 2018 data).
* We set the level of lump sum transfers at 40\% of labor income (roughly the case for 2018 data).
* Finally, *for now/temporarily*, adjust the value of *TotalTax* to ensure that the consumer has a balanced budget. If we used detailed data on regulation on households, we could drop this assumption.

In [52]:
output = adj.rc_pd(db.get('qS'), db['s_HH'])
inputs = adj.rc_pd(db.get('qD'), db['s_HH'])
rTransfer = 0.4004
rLaborTax = 0.478
rVAT = adj.rc_pd(db.get('vTax').xs('VAT',level='taxTypes'), db['s_HH']) / pyDatabases.pdSum(inputs, 'n')
gpyDB.add_or_merge_vals(db, adjMultiIndex.applyMult(rVAT,inputs.index), name = 'tauD') # add flat rate VAT on all demand
gpyDB.add_or_merge_vals(db, pd.Series(rLaborTax, index = output.index, name = 'tauS'), name = 'tauS') # add a flat labor tax rate
gpyDB.add_or_merge_vals(db, pyDatabases.pdSum(-output * rTransfer, 'n').rename('tauLump'), name = 'tauLump') # Adjust lump sum tax
gpyDB.add_or_merge_vals(db, pyDatabases.pdSum(rVAT * inputs, 'n')+pyDatabases.pdSum(output, 'n') * (rLaborTax-rTransfer), name = 'TotalTax') # adhoc adjustment of 'TotalTax'

## 4. Government

### 4.1. Nesting structure

Government consumption is nested in a similar way as household consumption - and uses the same elasticities:

In [53]:
dImport = adj.rc_pd(db.get('dImport'), db.get('s_G'))
dImport_dom = adj.rc_pd(db.get('dImport_dom'), db.get('s_G'))
dImport_for = adj.rc_pd(db.get('dImport_for'), db.get('s_G'))

*Add mappings:*

In [54]:
df = dImport.to_frame(index=False).assign(m = lambda x: 'GC_'+x['n'].astype(str), u = 'GC')
m = pd.MultiIndex.from_frame(df[['s','u','m']]).rename(['s','n','nn']) # mapping from materials to materials_x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','n']]).rename(['s','n','nn'])) # mapping from materials_x to x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','nn']]).rename(['s','n','nn'])) # mapping from materials_x to x_F

*Domestic only*

In [55]:
df = dImport_dom.to_frame(index=False).assign(u = 'GC')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Foreign only:

In [56]:
df = dImport_for.to_frame(index=False).assign(u = 'GC')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Replace the upper-most level with the name of the household consumption aggregate:

In [57]:
df = m.to_frame(index=False)
df.loc[df.n == 'GC','n'] = 'GC_'+df.loc[df.n == 'GC', 's']
m = pd.MultiIndex.from_frame(df)

*Add nesting structure:*

In [58]:
db['nestG'] = m

### 4.2. Elasticities

Upper level nest:

In [59]:
sigma_G_upper = pd.Series(0.3, index = m[m.get_level_values('n') == 'GC_G'].droplevel('nn').unique(), name = 'sigma')

Lower-level (import/domestic competition):

In [60]:
sigma_G_Import = pd.Series(2, index = m[m.get_level_values('n') != 'GC_G'].droplevel('nn').unique(), name = 'sigma')

Add to database:

In [61]:
sigma_G = pd.concat([sigma_G_upper, sigma_G_Import], axis = 0) 
gpyDB.add_or_merge_vals(db, sigma_G, name = 'sigma')

### 4.2. Regulation

The government regulates itself (because data says so). Because we use incomplete data on taxes, we make some adhoc adjustments here, though. More specifically, we go through the steps:
* Remove taxes on inventory (because the "inventory sector" is only used for completeness of the IO system).
* Set flat VAT tax rate on government consumption to target taxes from government sector.

In [62]:
inputs = adj.rc_pd(db.get('qD'), db.get('s_G'))
db['TotalTax'] = adj.rc_pd(db.get('TotalTax'), ('not', pd.Index(['itory'],name='s'))) # remove inventory taxes
db['d_TotalTax'] = db['TotalTax'].index # what sectors pay taxes
rVAT = adj.rc_pd(db.get('TotalTax'), db.get('s_G')) / pyDatabases.pdSum(inputs,'n') # define VAT rate of taxes
gpyDB.add_or_merge_vals(db, adjMultiIndex.applyMult(rVAT,inputs.index), name = 'tauD') # add flat rate VAT on all demand

## Export

In [63]:
db.export(repo=d['processedData'])